<a href="https://colab.research.google.com/github/olivia-sp/smart-sv-dev/blob/main/ssv_getRoadPOI_v0_0_olivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#도로정보DB - https://data.gg.go.kr/portal/data/dataset/searchDatasetPage.do
#GPS 데이터 기반 연산 로직 개발
#ver0.1 - 현재 위치 반경 계산 및 도로정보 필터링_신호등위치


In [1]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


#DB(csv)파일읽어오기

##신호등위치(성남시분당구)

In [41]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/신호등현황(개방표준).csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/신호등현황(개방표준).csv

% cd /content


already
/content


In [56]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/신호등현황(개방표준).csv'



In [57]:
raw_data = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data.info()
raw_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2972 entries, 9 to 9
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시군구명           2972 non-null   object 
 1   도로종류           2972 non-null   object 
 2   도로노선번호         1430 non-null   object 
 3   도로노선명          2972 non-null   object 
 4   도로노선방향         2972 non-null   int64  
 5   소재지도로명주소       564 non-null    object 
 6   소재지지번주소        1420 non-null   object 
 7   위도             1420 non-null   float64
 8   경도             1420 non-null   float64
 9   신호기설치방식        2972 non-null   int64  
 10  도로형태           2972 non-null   int64  
 11  주도로여부          2972 non-null   object 
 12  신호등관리번호        2972 non-null   object 
 13  신호등구분          2972 non-null   int64  
 14  신호등색종류         2972 non-null   int64  
 15  신호등화방식         2972 non-null   int64  
 16  신호등화순서         2972 non-null   object 
 17  신호등화시간         2972 non-null   object 
 18  광원종류       

시군구명                0
도로종류                0
도로노선번호           1542
도로노선명               0
도로노선방향              0
소재지도로명주소         2408
소재지지번주소          1552
위도               1552
경도               1552
신호기설치방식             0
도로형태                0
주도로여부               0
신호등관리번호             0
신호등구분               0
신호등색종류              0
신호등화방식              0
신호등화순서              0
신호등화시간              0
광원종류                0
신호제어방식              0
신호시간결정방식            0
점멸등운영여부             0
점멸등운영시작시각           0
점멸등운영종료시각           0
보행자작동신호기유무          0
잔여시간표시기유무           0
시각장애인용음향신호기유무       0
도로안내표지일련번호       2972
관리기관명               0
관리기관전화번호            0
데이터기준일자             0
dtype: int64

In [81]:
if 'index' not in raw_data.columns:
  raw_data.reset_index(inplace=True)

In [126]:
#공백제거
raw_data["시군구명"] = raw_data["시군구명"].str.replace(" ", "")
raw_data.head(3)

,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,...,점멸등운영여부,점멸등운영시작시각,점멸등운영종료시각,보행자작동신호기유무,잔여시간표시기유무,시각장애인용음향신호기유무,도로안내표지일련번호,관리기관명,관리기관전화번호,데이터기준일자
0,9,광명시,시도,NaN,목감로,3,NaN,NaN,NaN,NaN,...,Y,00:00,00:00,N,N,N,NaN,광명시 도시교통과,02-2680-2585,2022-04-29
1,9,광명시,시도,NaN,목감로,3,NaN,NaN,NaN,NaN,...,Y,00:00,00:00,N,N,N,NaN,광명시 도시교통과,02-2680-2585,2022-04-29
2,9,광명시,시도,NaN,목감로,3,NaN,NaN,NaN,NaN,...,Y,00:00,00:00,N,N,N,NaN,광명시 도시교통과,02-2680-2585,2022-04-29


In [127]:
filter_city=raw_data['시군구명'] == '성남시분당구'
df_SNcity = raw_data[filter_city]

In [128]:
df_SNcity.reset_index(inplace=True, drop=True)

In [129]:
df_SNcity.head(3)

,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,...,점멸등운영여부,점멸등운영시작시각,점멸등운영종료시각,보행자작동신호기유무,잔여시간표시기유무,시각장애인용음향신호기유무,도로안내표지일련번호,관리기관명,관리기관전화번호,데이터기준일자
0,9,성남시분당구,시도,판교원로 82번길,판교원로 82번길,3,경기도 성남시 분당구 판교원로82번길 60,경기도 성남시 분당구 운중동 916,37.384058,127.072437,...,Y,00:00,05:30,N,N,N,NaN,성남시청,031-729-3673,2018-08-31
1,9,성남시분당구,시도,판교역로,판교역로,3,경기도 성남시 분당구 판교역로 188,경기도 성남시 분당구 삼평동 642,37.397392,127.109988,...,N,00:00,00:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
2,9,성남시분당구,시도,분당로,분당로,3,NaN,경기도 성남시 분당구 분당동 195-7,37.367393,127.140821,...,N,00:00,00:00,N,N,N,NaN,성남시청,031-729-3673,2018-08-31


#fake GPS 더미

In [17]:
!pip3 install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
from faker import Faker
fake = Faker('ko_KR')
# for _ in range(5):
#   latlng = fake.local_latlng('KR')
#   print(latlng)

latlng = fake.local_latlng('KR')
print(latlng)
lat = float (latlng[0])
lon = float (latlng[1])
my_latlng = (lat, lon)
print(my_latlng)

('34.8825', '128.62667', 'Sinhyeon', 'KR', 'Asia/Seoul')
(34.8825, 128.62667)


# 현재 위치 반경 계산

In [9]:
!pip3 install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
### 공식문서 예시
# from haversine import haversine, Unit

# lyon = (45.7597, 4.8422) # (lat, lon)
# paris = (48.8567, 2.3508)

# haversine(lyon, paris)
# >> 392.2172595594006  # in kilometers

# haversine(lyon, paris, unit=Unit.MILES)
# >> 243.71201856934454  # in miles

# # you can also use the string abbreviation for units:
# haversine(lyon, paris, unit='mi')
# >> 243.71201856934454  # in miles

# haversine(lyon, paris, unit=Unit.NAUTICAL_MILES)
# >> 211.78037755311516  # in nautical miles

In [89]:
df_SNcity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 348 to 1027
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시도명            414 non-null    int64  
 1   시군구명           414 non-null    object 
 2   도로종류           414 non-null    object 
 3   도로노선번호         414 non-null    object 
 4   도로노선명          414 non-null    object 
 5   도로노선방향         414 non-null    int64  
 6   소재지도로명주소       298 non-null    object 
 7   소재지지번주소        414 non-null    object 
 8   위도             414 non-null    float64
 9   경도             414 non-null    float64
 10  신호기설치방식        414 non-null    int64  
 11  도로형태           414 non-null    int64  
 12  주도로여부          414 non-null    object 
 13  신호등관리번호        414 non-null    object 
 14  신호등구분          414 non-null    int64  
 15  신호등색종류         414 non-null    int64  
 16  신호등화방식         414 non-null    int64  
 17  신호등화순서         414 non-null    object 
 18  신호등화시간 

In [140]:
#판교만도중앙연구소
my_latlng = (37.404704734328, 127.10535530866)

In [154]:
from haversine import haversine, Unit
distance_compare_list = []

## for문으로 비교
for i in range(df_SNcity.shape[0]):
    db_trafLgt = (df_SNcity["위도"][i], df_SNcity["경도"][i])
    result = haversine(db_trafLgt, my_latlng, unit='m')
    if result <= 500:
      distance_compare_list.append(result)
      print(db_trafLgt)
#print(distance_compare_list)

(37.4037259458, 127.0998775385)
(37.4045092638, 127.1005562139)
(37.4041629952, 127.1051525762)
(37.4044202324, 127.1097384586)
(37.4033105416, 127.1010177224)
(37.4033105416, 127.1010177224)
(37.4019750211, 127.1034329076)
(37.4045044843, 127.1061808333)
(37.400611472, 127.1069321778)
(37.4033935026, 127.1092739361)
(37.4027630214, 127.1090245731)
